#Feeling Reality Data Analysis


In [ ]:
!python --version

## Mount Google Drive

In [ ]:
from google.colab import drive #mount Drive
drive.mount('/content/drive')

## Import Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import matplotlib.pyplot as plt

## Read in Data

In [ ]:
df=pd.read_excel('<path>') #comment out and import from your own path as needed

In [ ]:
df

## Preprocessing

Subsetting Demographics

In [ ]:
df = df.rename(columns={'post_Q23': 'age', 'post_Q24':'gender', 'post_Q25':'race_ethnicity'})
df

In [ ]:
demographics_columns = ['ID', 'condition_order', 'age', 'gender', 'race_ethnicity']
demographics_df = df[demographics_columns]
demographics_df

In [ ]:
demographics_df['age'].value_counts()

In [ ]:
demographics_df['gender'].value_counts()

In [ ]:
demographics_df['race_ethnicity'].value_counts()

In [ ]:
df.columns

Subsetting pre, during, & post

In [ ]:
#pre
pre_columns = ['ID', 'condition_order', 'pre_Q1', 'pre_Q2', 'pre_Q3', 'pre_Q4', 'pre_Q5', 'pre_Q6','pre_Q7', 'pre_Q8', 'pre_Q9', 'pre_Q10']
pre_df = df[pre_columns]
pre_df

In [ ]:
pre_rename_mapping = {f"pre_Q{i}": f"pre_RSE_{i}" for i in range(1, 11)}

# Rename the columns
pre_df = pre_df.rename(columns=pre_rename_mapping)
pre_df

In [ ]:
#during
during_columns = ['ID', 'condition_order', 'during_Q1', 'during_Q2', 'during_Q3', 'during_Q4', 'during_Q5','during_Q6', 'during_Q7', 'during_Q8', 'during_Q9', 'during_Q10', 'during_Q11', 'during_Q12',
                  'during_Q13', 'during_Q14', 'during_Q15', 'during_Q16', 'during_Q17', 'during_Q18', 'during_Q19', 'during_Q20', 'during_Q21', 'during_Q22']
during_df = df[during_columns]
during_df

In [ ]:
#post
post_columns = ['ID', 'condition_order','post_Q1', 'post_Q2', 'post_Q3', 'post_Q4',
       'post_Q5', 'post_Q6', 'post_Q7', 'post_Q8', 'post_Q9', 'post_Q10',
       'post_Q11', 'post_Q12', 'post_Q13', 'post_Q14', 'post_Q15', 'post_Q16',
       'post_Q17', 'post_Q18', 'post_Q19', 'post_Q20', 'post_Q21', 'post_Q22']
post_df = df[post_columns]
post_df

Subsetting mirror and non-mirror based on condition order from during and post dfs

In [ ]:
mirror_df = pd.DataFrame()
noMirror_df = pd.DataFrame()

In [ ]:
for index, row in df.iterrows():
    try:
        if row['condition_order'] == 'm1':
            during_row = during_df.loc[[index]].copy()
            during_row['ID'] = row['ID']
            during_row['condition_order'] = row['condition_order']
            mirror_df = pd.concat([mirror_df, during_row], ignore_index=True)

            post_row = post_df.loc[[index]].copy()
            post_row['ID'] = row['ID']
            post_row['condition_order'] = row['condition_order']
            noMirror_df = pd.concat([noMirror_df, post_row], ignore_index=True)

        elif row['condition_order'] == 'm2':
            during_row = during_df.loc[[index]].copy()
            during_row['ID'] = row['ID']
            during_row['condition_order'] = row['condition_order']
            noMirror_df = pd.concat([noMirror_df, during_row], ignore_index=True)

            post_row = post_df.loc[[index]].copy()
            post_row['ID'] = row['ID']
            post_row['condition_order'] = row['condition_order']
            mirror_df = pd.concat([mirror_df, post_row], ignore_index=True)
    except KeyError as e:
        print(f"Missing index in during_df or post_df: {e}")
        continue

In [ ]:
mirror_df.head()

In [ ]:
noMirror_df.head()

Within each subset, creating during and post subsets:
* mirror_during
* mirror_post
* noMirror_during
* noMirror_post

In [ ]:
#mirror_during
mirror_during = mirror_df[during_columns]
mirror_during

In [ ]:
#mirror_post
mirror_post = mirror_df[post_columns]
mirror_post

In [ ]:
#noMirror_during
noMirror_during = noMirror_df[during_columns]
noMirror_during

In [ ]:
#noMirror_post
noMirror_post = noMirror_df[post_columns]
noMirror_post

Intuitively renaming RSE & VEQ fields

In [ ]:
def rename_mirror_during_column(col):
    match = re.match(r"during_Q(\d+)", col)
    if match:
        question_number = int(match.group(1))
        if question_number <= 12:
            return f"mirror_VEQ_{question_number}"
        else:
            return f"mirror_RSE_{question_number - 12}"
    return col

mirror_during = mirror_during.rename(columns=rename_mirror_during_column)

In [ ]:
mirror_during.columns

In [ ]:
def rename_mirror_post_column(col):
    match = re.match(r"post_Q(\d+)", col)
    if match:
        question_number = int(match.group(1))
        if question_number <= 12:
            return f"mirror_VEQ_{question_number}"
        else:
            return f"mirror_RSE_{question_number - 12}"
    return col

mirror_post = mirror_post.rename(columns=rename_mirror_post_column)

In [ ]:
mirror_post.columns

In [ ]:
def rename_noMirror_during_column(col):
    match = re.match(r"during_Q(\d+)", col)
    if match:
        question_number = int(match.group(1))
        if question_number <= 12:
            return f"noMirror_VEQ_{question_number}"
        else:
            return f"noMirror_RSE_{question_number - 12}"
    return col

noMirror_during = noMirror_during.rename(columns=rename_noMirror_during_column)

In [ ]:
noMirror_during.columns

In [ ]:
def rename_noMirror_post_column(col):
    match = re.match(r"post_Q(\d+)", col)
    if match:
        question_number = int(match.group(1))
        if question_number <= 12:
            return f"noMirror_VEQ_{question_number}"
        else:
            return f"noMirror_RSE_{question_number - 12}"
    return col

noMirror_post = noMirror_post.rename(columns=rename_noMirror_post_column)

In [ ]:
noMirror_post.columns

Merge complement dfs:
* mirror_during + mirror_post = `mirror`


* noMirror_during + noMirror_post = `noMirror`

In [ ]:
mirror = mirror_during.combine_first(mirror_post)
mirror

In [ ]:
noMirror = noMirror_during.combine_first(noMirror_post)
noMirror

Checking column names

In [ ]:
pre_df.columns

In [ ]:
mirror.columns

In [ ]:
noMirror.columns

###Numeric Mapping

**RSE**

Higher scores --> higher self-esteem

Lower scores --> lower self-esteem

**Default**
* "Strongly Disagree": 1
* "Disagree": 2,
* "Agree": 3,
* "Strongly Agree": 4

**Inverse**
* "Strongly Disagree": 4,
* "Disagree": 3,
* "Agree": 2,
* "Strongly Agree": 1

----------------------------------------

**VEQ**

Higher scores --> Grester virtual embodiment

Lower scores --> Lesser virtual embodiment

* Strongly Disagree: 1
* Disagree: 2
* Somewhat Disagree: 3
* Neither Agree nor Disagree: 4
* Somewhat Agree: 5
* Agree: 6
* Strongly Agree: 7


RSE encoding

In [ ]:
def encode_RSE_responses(df):
    #Default mapping
    default_mapping = {
        "Strongly Disagree": 1,
        "Disagree": 2,
        "Agree": 3,
        "Strongly Agree": 4
    }

    #Reverse mapping
    reverse_mapping = {
        "Strongly Disagree": 4,
        "Disagree": 3,
        "Agree": 2,
        "Strongly Agree": 1
    }

    #Inverted fields
    reverse_fields = [
        "pre_RSE_2", "pre_RSE_5", "pre_RSE_6", "pre_RSE_8",
        "pre_RSE_9", "mirror_RSE_2", "mirror_RSE_5",
        "mirror_RSE_6", "mirror_RSE_8", "mirror_RSE_9",
        "noMirror_RSE_2", "noMirror_RSE_5", "noMirror_RSE_6",
        "noMirror_RSE_8", "noMirror_RSE_9"
    ]

    def map_responses(col):
        if col.name in reverse_fields:
            return col.map(reverse_mapping).fillna(col)
        return col.map(default_mapping).fillna(col)

    #Only apply encoding to fields containing "RSE"
    return df.apply(lambda col: map_responses(col) if "RSE" in col.name else col)

In [ ]:
pre_df

In [ ]:
encoded_pre = encode_RSE_responses(pre_df)
encoded_pre

In [ ]:
mirror

In [ ]:
encoded_mirror = encode_RSE_responses(mirror)
encoded_mirror

In [ ]:
noMirror

In [ ]:
encoded_noMirror = encode_RSE_responses(noMirror)
encoded_noMirror

VEQ encoding

In [ ]:
def encode_VEQ_responses(df):

    mapping = {
        "Strongly Disagree": 1,
        "Disagree": 2,
        "Somewhat Disagree": 3,
        "Neither Agree nor Disagree": 4,
        "Somewhat Agree": 5,
        "Agree": 6,
        "Strongly Agree": 7
    }

    def map_responses(col):
        return col.map(mapping).fillna(col)

    #Only apply encoding to fields containing "VEQ"
    return df.apply(lambda col: map_responses(col) if "VEQ" in col.name else col)

In [ ]:
encoded_mirror

In [ ]:
encoded_mirror = encode_VEQ_responses(encoded_mirror)
encoded_mirror

In [ ]:
encoded_noMirror

In [ ]:
encoded_noMirror = encode_VEQ_responses(encoded_noMirror)
encoded_noMirror

In [ ]:
encoded_pre.head()

In [ ]:
encoded_mirror.head()

In [ ]:
encoded_noMirror.head()

## Analysis

### Pre-Assessment

#### RSE

##### By Question

In [ ]:
pre_columns = ['pre_RSE_1', 'pre_RSE_2', 'pre_RSE_3', 'pre_RSE_4', 'pre_RSE_5', 'pre_RSE_6', 'pre_RSE_7', 'pre_RSE_8', 'pre_RSE_9', 'pre_RSE_10']

for col in pre_columns:
    print(f"Mean {col}: {encoded_pre[col].mean()}")

##### By Participant

In [ ]:
#row averages
encoded_pre['row_mean'] = encoded_pre[pre_columns].mean(axis=1)

print(encoded_pre[['ID', 'row_mean']])

##### Overall

In [ ]:
overall_pre_average = sum(encoded_pre[col].mean() for col in pre_columns) / len(pre_columns)
print(f"Overall RSE average for Pre-assessment (column-wise mean): {overall_pre_average}")

### With Mirror

#### RSE

##### By Question

In [ ]:
mirror_rse_columns = ['mirror_RSE_1', 'mirror_RSE_2', 'mirror_RSE_3', 'mirror_RSE_4', 'mirror_RSE_5', 'mirror_RSE_6', 'mirror_RSE_7', 'mirror_RSE_8', 'mirror_RSE_9', 'mirror_RSE_10']

for col in mirror_rse_columns:
    print(f"Mean {col}: {encoded_mirror[col].mean()}")

#####By Participant

In [ ]:
#row averages
encoded_mirror['row_mean'] = encoded_mirror[mirror_rse_columns].mean(axis=1)

print(encoded_mirror[['ID', 'row_mean']])

##### Overall

In [ ]:
overall_mirror_average = sum(encoded_mirror[col].mean() for col in mirror_rse_columns) / len(mirror_rse_columns)
print(f"Overall RSE average for Mirror Condition (column-wise mean): {overall_mirror_average}")

#### VEQ

##### By Question

In [ ]:
mirror_veq_columns = ['mirror_VEQ_1', 'mirror_VEQ_2', 'mirror_VEQ_3',
       'mirror_VEQ_4', 'mirror_VEQ_5', 'mirror_VEQ_6', 'mirror_VEQ_7',
       'mirror_VEQ_8', 'mirror_VEQ_9', 'mirror_VEQ_10', 'mirror_VEQ_11',
       'mirror_VEQ_12']

for col in mirror_veq_columns:
    print(f"Mean {col}: {encoded_mirror[col].mean()}")

##### By Participant

In [ ]:
#row averages
encoded_mirror['row_mean'] = encoded_mirror[mirror_veq_columns].mean(axis=1)

print(encoded_mirror[['ID', 'row_mean']])

##### Overall

In [ ]:
overall_mirror_veq_average = sum(encoded_mirror[col].mean() for col in mirror_veq_columns) / len(mirror_veq_columns)
print(f"Overall VEQ average for Mirror Condition (column-wise mean): {overall_mirror_veq_average}")

### Without Mirror

#### RSE

##### By Question

In [ ]:
noMirror_rse_columns = ['noMirror_RSE_1', 'noMirror_RSE_2', 'noMirror_RSE_3', 'noMirror_RSE_4', 'noMirror_RSE_5', 'noMirror_RSE_6', 'noMirror_RSE_7', 'noMirror_RSE_8', 'noMirror_RSE_9', 'noMirror_RSE_10']

for col in noMirror_rse_columns:
    print(f"Mean {col}: {encoded_noMirror[col].mean()}")

##### By Participant

In [ ]:
#row averages
encoded_noMirror['row_mean'] = encoded_noMirror[noMirror_rse_columns].mean(axis=1)

print(encoded_noMirror[['ID', 'row_mean']])

##### Overall

In [ ]:
overall_noMirror_average = sum(encoded_noMirror[col].mean() for col in noMirror_rse_columns) / len(noMirror_rse_columns)
print(f"Overall RSE average for No Mirror Condition (column-wise mean): {overall_noMirror_average}")

####VEQ

##### By Question

In [ ]:
noMirror_veq_columns = ['noMirror_VEQ_1', 'noMirror_VEQ_2',
       'noMirror_VEQ_3', 'noMirror_VEQ_4', 'noMirror_VEQ_5', 'noMirror_VEQ_6',
       'noMirror_VEQ_7', 'noMirror_VEQ_8', 'noMirror_VEQ_9', 'noMirror_VEQ_10',
       'noMirror_VEQ_11', 'noMirror_VEQ_12']

for col in noMirror_veq_columns:
    print(f"Mean {col}: {encoded_noMirror[col].mean()}")

##### By Participant

In [ ]:
#row averages
encoded_noMirror['row_mean'] = encoded_noMirror[noMirror_veq_columns].mean(axis=1)

print(encoded_noMirror[['ID', 'row_mean']])

##### Overall

In [ ]:
overall_noMirror_veg_average = sum(encoded_noMirror[col].mean() for col in noMirror_veq_columns) / len(noMirror_veq_columns)
print(f"Overall VEQ average for No Mirror Condition (column-wise mean): {overall_noMirror_veg_average}")

Figures

In [ ]:
#Participant RSE Line Graph

rse_avg_data = {
    "Participant": ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"],
    "Baseline": [3.2, 2.4, 2.5, 3.0, 3.7, 3.6, 3.8, 3.7, 3.1, 3.1],
    "Without Mirror": [3.6, 3.2, 3.7, 3.7, 3.3, 3.2, 3.3, 2.2, 3.4, 3.5],
    "With Mirror": [3.0, 2.8, 3.2, 3.1, 3.7, 3.3, 3.8, 3.7, 3.7, 2.9]
}

rse_avg_df = pd.DataFrame(rse_avg_data)
rse_avg_df

In [ ]:
conditions = ["Baseline", "Without Virtual Mirror", "With Virtual Mirror"]
x_positions = range(len(conditions))

plt.figure(figsize=(10, 6))
for _, row in rse_avg_df.iterrows():
    scores = [row["Baseline"], row["Without Mirror"], row["With Mirror"]]
    plt.plot(x_positions, scores, marker='o', label=row["Participant"])

plt.xticks(x_positions, conditions)
plt.xlabel("Condition")
plt.ylabel("Average RSE Score")
plt.title("Participant Self-Esteem Score at Baseline & With/Without Virtual Mirror (By Participant)")
plt.legend(title="Participant", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig("high_quality_linePlot.png", dpi=500, bbox_inches='tight')
plt.show()

In [ ]:
#Boxplots

plt.figure(figsize=(8, 6))

data = [rse_avg_df["Baseline"], rse_avg_df["Without Mirror"], rse_avg_df["With Mirror"],]
labels = ["Baseline", "Without Virtual Mirror", "With Virtual Mirror"]

plt.boxplot(data, labels=labels, patch_artist=True, boxprops=dict(facecolor="purple"))

plt.title("Participant Self-Esteem Score at Baseline & With/Without Virtual Mirror (Overall)")
plt.ylabel("Average RSE Score")
plt.grid(axis="y", linestyle="--", alpha=0.5)

plt.tight_layout()
plt.savefig("high_quality_boxPlot.png", dpi=500, bbox_inches='tight')
plt.show()

In [ ]:
participants = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10']
veq_without_mirror = [4.5, 6.3, 3.3, 3.7, 4.3, 6.2, 5.3, 3.7, 4.8, 4.7]
veq_with_mirror = [4.6, 6.3, 4.8, 4.1, 2.8, 4.8, 5.6, 2.8, 2.2, 5.5]

x = np.arange(len(participants))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, veq_without_mirror, width, label='Without Virtual Mirror', color='skyblue')
bars2 = ax.bar(x + width/2, veq_with_mirror, width, label='With Virtual Mirror', color='orange')
plt.grid(axis="y", linestyle="--", alpha=0.7)

ax.set_xlabel('Participants')
ax.set_ylabel('Average VEQ Scores')
ax.set_title('Average Virtual Embodiment Scores by Participant')
ax.set_xticks(x)
ax.set_xticklabels(participants)
ax.legend()

plt.tight_layout()
plt.savefig("high_quality_barPlot.png", dpi=500, bbox_inches='tight')
plt.show()

In [ ]:
veq_avg_data = {
    "Participant": ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"],
    "Without Mirror": [4.5, 6.3, 3.3, 3.7, 4.3, 6.2, 5.3, 3.7, 4.8, 4.7],
    "With Mirror": [4.6, 6.3, 4.8, 4.1, 2.8, 4.8, 5.6, 2.8, 2.2, 5.5]
}

veq_avg_data = pd.DataFrame(veq_avg_data)
veq_avg_data

In [ ]:
#Boxplots

plt.figure(figsize=(8, 6))

data = [veq_avg_data["Without Mirror"], rse_avg_df["With Mirror"],]
labels = ["Without Virtual Mirror", "With Virtual Mirror"]

plt.boxplot(data, labels=labels, patch_artist=True, boxprops=dict(facecolor="purple"))

plt.title("Participant Virtual Embodiment Score With & Without Virtual Mirror")
plt.ylabel("Average VEQ Score")
plt.grid(axis="y", linestyle="--", alpha=0.5)

plt.tight_layout()
plt.savefig("high_quality_veq_boxPlot.png", dpi=500, bbox_inches='tight')
plt.show()

In [ ]:
conditions = ["Without Virtual Mirror", "With Virtual Mirror"]
x_positions = range(len(conditions))

plt.figure(figsize=(10, 6))
for _, row in veq_avg_data.iterrows():
    scores = [row["Without Mirror"], row["With Mirror"]]
    plt.plot(x_positions, scores, marker='o', label=row["Participant"])

plt.xticks(x_positions, conditions)
plt.xlabel("Condition")
plt.ylabel("Average VEQ Score")
plt.title("Participant Virtual Embodiment Score With and Without Virtual Mirror")
plt.legend(title="Participant", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig("high_quality_veq_linePlot.png", dpi=500, bbox_inches='tight')
plt.show()

Tables

In [ ]:
#Score by question

rse_avg_data_q = {
    "Question": ["Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q7", "Q8", "Q9", "Q10"],
    "Baseline": [3.5, 2.9, 3.4, 3.5, 3.2, 3.0, 3.2, 2.2, 3.7, 3.5],
    "Without Mirror": [3.6, 3.2, 3.7, 3.7, 3.3, 3.2, 3.3, 2.2, 3.4, 3.5],
    "With Mirror": [3.3, 3.3, 3.6, 3.7, 3.3, 3.2, 3.4, 2.0, 3.7, 3.7]
}

rse_avg_q_df = pd.DataFrame(rse_avg_data_q)

rse_avg_q_df['% Change Baseline to Without Mirror'] = ((rse_avg_q_df['Without Mirror'] - rse_avg_q_df['Baseline']) / rse_avg_q_df['Baseline']) * 100
rse_avg_q_df['% Change Baseline to With Mirror'] = ((rse_avg_q_df['With Mirror'] - rse_avg_q_df['Baseline']) / rse_avg_q_df['Baseline']) * 100
rse_avg_q_df['% Difference Without vs With Mirror'] = ((rse_avg_q_df['Without Mirror'] - rse_avg_q_df['With Mirror']).abs() / ((rse_avg_q_df['Without Mirror'] + rse_avg_q_df['With Mirror']) / 2)) * 100

rse_avg_q_df


In [ ]:
#Score by participant

rse_avg_data = {
    "Participant": ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"],
    "Baseline": [3.2, 2.4, 2.5, 3.0, 3.7, 3.6, 3.8, 3.7, 3.1, 3.1],
    "Without Mirror": [3.6, 3.2, 3.7, 3.7, 3.3, 3.2, 3.3, 2.2, 3.4, 3.5],
    "With Mirror": [3.0, 2.8, 3.2, 3.1, 3.7, 3.3, 3.8, 3.7, 3.7, 2.9]
}

rse_avg_df = pd.DataFrame(rse_avg_data)

rse_avg_df['% Change Baseline to Without Mirror'] = ((rse_avg_df['Without Mirror'] - rse_avg_df['Baseline']) / rse_avg_df['Baseline']) * 100
rse_avg_df['% Change Baseline to With Mirror'] = ((rse_avg_df['With Mirror'] - rse_avg_df['Baseline']) / rse_avg_df['Baseline']) * 100
rse_avg_df['% Difference Without vs With Mirror'] = ((rse_avg_df['Without Mirror'] - rse_avg_df['With Mirror']).abs() / ((rse_avg_df['Without Mirror'] + rse_avg_df['With Mirror']) / 2)) * 100

rse_avg_df

In [ ]:
#VEQ % diff

VEQ_avg_data = {
    "Participant": ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10"],
    "Without Mirror": [4.5, 6.3, 3.3, 3.7, 4.3, 6.2, 5.3, 3.7, 4.8, 4.7],
    "With Mirror": [4.6, 6.3, 4.8, 4.1, 2.8, 4.8, 5.6, 2.8, 2.2, 5.5]
}

veq_avg_df = pd.DataFrame(VEQ_avg_data)

veq_avg_df['% Difference Without vs With Mirror'] = ((veq_avg_df['Without Mirror'] - veq_avg_df['With Mirror']).abs() / ((veq_avg_df['Without Mirror'] + veq_avg_df['With Mirror']) / 2)) * 100

veq_avg_df

##Wilcoxon signed-rank test

### Self Esteem

In [ ]:
from scipy.stats import wilcoxon

In [ ]:
baseline = [3.2, 2.4, 2.5, 3.0, 3.7, 3.6, 3.8, 3.7, 3.1, 3.1]
without_mirror = [3.6, 3.2, 3.7, 3.7, 3.3, 3.2, 3.3, 2.2, 3.4, 3.5]
with_mirror = [3.0, 2.8, 3.2, 3.1, 3.7, 3.3, 3.8, 3.7, 3.7, 2.9]

In [ ]:
stat, p = wilcoxon(baseline, without_mirror)

print(f"Test Statistic: {stat}")
print(f"P-Value: {p}")


In [ ]:
stat, p = wilcoxon(baseline, with_mirror)

print(f"Test Statistic: {stat}")
print(f"P-Value: {p}")

In [ ]:
stat, p = wilcoxon(without_mirror, with_mirror)

print(f"Test Statistic: {stat}")
print(f"P-Value: {p}")

### Virtual Embodiment

In [ ]:
veq_without_mirror = [4.5, 6.3, 3.3, 3.7, 4.3, 6.2, 5.3, 3.7, 4.8, 4.7]
veq_with_mirror = [4.6, 6.3, 4.8, 4.1, 2.8, 4.8, 5.6, 2.8, 2.2, 5.5]

In [ ]:
stat, p = wilcoxon(veq_without_mirror, veq_with_mirror)

print(f"Test Statistic: {stat}")
print(f"P-Value: {p}")
